In [ ]:
import brainsss
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#from sklearn.cluster import AgglomerativeClustering
import scipy
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from scipy.cluster import hierarchy
from scipy.signal import butter, filtfilt, freqz
import matplotlib as mpl
from matplotlib.pyplot import cm
import random
from scipy.stats import sem
import time
import h5py
import ants
import nibabel as nib
import matplotlib
from scipy.ndimage import gaussian_filter1d
import pickle

In [ ]:
fly_num = 'fly_194'
func_path = f'/oak/stanford/groups/trc/data/Ilana/2P/data/{fly_num}/'
# func_path = f'/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/{fly_num}/'


In [ ]:
###########################
### PREP VISUAL STIMULI ###
###########################

vision_path = os.path.join(func_path,'func_0', 'visual')

### Load Photodiode ###
t, ft_triggers, pd1, pd2 = brainsss.load_photodiode(vision_path)
stimulus_start_times = brainsss.extract_stim_times_from_pd(pd1, t)

# *100 puts in units of 10ms, which will match fictrac
st_10ms = [int(stimulus_start_times[i]*100) for i in range(len(stimulus_start_times))]

# get 1ms version to match neural timestamps
st_ms= [i*10 for i in st_10ms]
starts_loom = st_10ms

In [ ]:
stimulus_start_times

In [ ]:
####################
### Prep Fictrac ###
####################

fictrac_path = os.path.join(func_path, 'func_0', 'fictrac')
fictrac_raw = brainsss.load_fictrac(fictrac_path)

fps = 100
resolution = 10 #desired resolution in ms
expt_len = fictrac_raw.shape[0]/fps*1000
behaviors = ['dRotLabY', 'dRotLabZ', 'dRotLabX']
fictrac = {}
for behavior in behaviors:
    if behavior == 'dRotLabY': short = 'Y'
    elif behavior == 'dRotLabZ': short = 'Z'
    elif behavior == 'dRotLabX': short = 'X'
    fictrac[short] = brainsss.smooth_and_interp_fictrac(fictrac_raw, fps, resolution, expt_len, behavior)
fictrac_timestamps = np.arange(0,expt_len,resolution)

In [ ]:
fictrac_raw.keys()

In [ ]:
#different way to smooth behavior

# def smooth(x, axis=0, wid=5):
#     # this is way faster than convolve, Minny smooth
#     if wid < 2:
#         return x
#     cumsum_vec = np.cumsum(np.insert(x, 0, 0, axis=axis), axis=axis)
#     ma_vec = (cumsum_vec[wid:] - cumsum_vec[:-wid]) / wid
#     y = x.copy()
#     start_ind = int(np.floor((wid-1)/2))
#     end_ind = wid-1-start_ind
#     y[start_ind:-end_ind] = ma_vec
#     return y

# def smooth_vel(X, Y, theta, width=0):
#     # Alex smooth
#     N = np.size(X)
    
#     sX = smooth(X, axis=0, wid=width)
#     sY = smooth(Y, axis=0, wid=width)
    
    
#     Fwd = np.zeros((N), dtype=float)
#     Slid = np.zeros((N), dtype=float)
    
#     for i in range(1,N):
#         dx = sX[i]-sX[i-1]
#         dy = sY[i]-sY[i-1]
#         tz = theta[i-1]
#         Fwd[i] = dx*np.cos(tz) + dy*np.sin(tz)
#         Slid[i] = dx*np.sin(tz) - dy*np.cos(tz)
#     return Fwd, Slid

In [ ]:
def extract_traces(fictrac, stim_times, pre_window, post_window, val=None):
    traces = []
    for i in range(len(stim_times)):
        if val != None:
            trace = fictrac[val][stim_times[i]-pre_window:stim_times[i]+post_window]
        else:
            trace = fictrac[stim_times[i]-pre_window:stim_times[i]+post_window]
        if len(trace) == pre_window + post_window: # this handles fictrac that crashed or was aborted or some bullshit
            traces.append(trace)
    traces = np.asarray(traces)
    mean_trace = np.mean(traces,axis=0)
    sem_trace = scipy.stats.sem(traces,axis=0)
    return traces, mean_trace, sem_trace

In [ ]:
#next two cells made for 10min fictrac cut off

weird_size = np.size(fictrac['Y'])
weird_idx = list(np.where(np.asarray(starts_loom) <= weird_size)[0])
actual_looms = np.asarray(starts_loom)[weird_idx]

actual_looms_ms = [i*10 for i in actual_looms]

In [ ]:
plt.plot(fictrac['Y'])
for i in range(len(actual_looms)):
    plt.axvline(actual_looms[i], color='r')
# plt.xlim(starts_loom[0], starts_loom[10])

In [ ]:
#QC to check if flies actually responding to loom
max_time = np.max(actual_looms)
min_time = 0
null_times = np.random.randint(min_time, max_time, len(actual_looms))
# this replaces starts_loom

In [ ]:
###########################################
### Extract Stimulus Triggered Behavior ###
###########################################

pre_window = 200
post_window = 300
# avg_around = 20
# stim_time = 75
# window = np.arange(-pre_window,post_window)

behavior_traces = {}
mean_trace = {}
sem_trace = {}
behavior_traces,mean_trace,sem_trace = extract_traces(fictrac['Y'], actual_looms, pre_window, post_window)
behavior_traces_QC,mean_trace_QC,sem_trace_QC = extract_traces(fictrac['Y'], null_times, pre_window, post_window)
# behavior_traces_x,mean_trace,sem_trace = extract_traces(fictrac, starts_loom, pre_window, post_window, 'X')
# behavior_traces_z,mean_trace,sem_trace = extract_traces(fictrac, starts_loom, pre_window, post_window, 'Z')


In [ ]:
plt.figure(figsize=(10,10))
plt.plot(mean_trace,color='k',linewidth=3)
plt.fill_between(np.arange(len(mean_trace)),mean_trace-sem_trace, mean_trace+sem_trace, color='k',alpha=0.3)
plt.axvline(200,color='k',linestyle='--',lw=2)
plt.axvline(300,color='k',linestyle='--',lw=2)
plt.ylim(-1, 5);

#null_hypothesis check
# plt.plot(mean_trace_QC,color='b',linewidth=3)
# plt.fill_between(np.arange(len(mean_trace_QC)),mean_trace_QC-sem_trace_QC, mean_trace_QC+sem_trace_QC, color='b',alpha=0.3)

In [ ]:
# np.shape(behavior_traces)
# rand_time_e = 250
# rand_time_s = 210
# diffs = behavior_traces[:,rand_time_e]-behavior_traces[:,rand_time_s]
# diffs_QC = behavior_traces_QC[:,rand_time_e]-behavior_traces_QC[:,rand_time_s]
# plt.subplot(2, 1, 1)
# plt.hist(diffs)
# plt.subplot(2, 1, 2)
# plt.hist(diffs_QC)
# # a randomly chosen location after loom to compare the speeds--consider changing this val

In [ ]:
ts = fictrac_raw["timeStamp"] #in ns i think
ts_sec = ts/1e9
fr = 1/np.mean(np.diff(ts_sec)) #figure out frame_rate from bruker fictrac

In [ ]:
len(ts_sec)

In [ ]:
fr

In [ ]:
# andrew anaylsis
value_struct ={
    # the data is in 10ms, use this value to get the index if i use seconds
    "tp_width_sec": 0.01,
    "trial_time_sec": 1,
    "before_stim_sec": pre_window/100,
    # time in seconds between end of pre trial window and the stimulus time. 
    # when set to zero, the pre trial window ends when the stimulus is presented
    "pre_trial_window_sec": -0.05,
    # size of window that pre trial speed is averaged over, seconds, window ends at stimulus time
    # when 0 is a single val and not a window
    "pre_trial_size_sec": 0,
    # size of post trial speed average window, sec
    "post_trial_window_sec": 0.5,
    # post trial window at stim tiem + post trail delay
    # when 0 is a single val and not a window
    "post_trial_size_sec": 0,
    "thresh": 0.05
    }
def get_vals_for_analysis(tp_width_sec=0, trial_time_sec=0, before_stim_sec=0, pre_trial_window_sec=0, pre_trial_size_sec=0, post_trial_window_sec=0, post_trial_size_sec=0, thresh=0):
        
    stim_end_sec = before_stim_sec + trial_time_sec
    
    stim_idx = int(before_stim_sec/tp_width_sec)
    stim_end_idx = int(stim_end_sec/tp_width_sec)
    pre_trial_window_idx = int(pre_trial_window_sec/tp_width_sec)

    # first val i use to make delta to compare change in vel
    first_val_end_idx = int(stim_idx - pre_trial_window_idx)
    first_val_start_idx = int(first_val_end_idx - (pre_trial_size_sec/tp_width_sec)) #only needed if you're averaging across a window, otherwise just start val is used

    # second val used to make delta
    second_val_start_idx = int(stim_idx + post_trial_size_sec/tp_width_sec)
    second_val_end_idx = int(second_val_start_idx + post_trial_window_sec/tp_width_sec)
    return stim_idx, stim_end_idx, pre_trial_window_idx, first_val_end_idx, first_val_start_idx, second_val_end_idx, second_val_start_idx

In [ ]:
def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data, method="gust")
    return y

In [ ]:
def apply_butter_lowpass(behavior_traces, stim_idx, fr):
    # Filter requirements.
    order = 4
    fs = fr      # sample rate, Hz
    cutoff = 3  # desired cutoff frequency of the filter, Hz

    # Get the filter coefficients so we can check its frequency response.
    b, a = butter_lowpass(cutoff, fs, order)

    # Plot the frequency response.
    w, h = freqz(b, a, fs=fs, worN=8000)
    plt.subplot(2, 1, 1)
    plt.plot(w, np.abs(h), 'b')
    plt.plot(cutoff, 0.5*np.sqrt(2), 'ko')
    plt.axvline(cutoff, color='k')
    plt.xlim(0, 0.5*fs)
    plt.title("Lowpass Filter Frequency Response")
    plt.xlabel('Frequency [Hz]')
    # plt.grid()

    # Filter the data, and plot both the original and filtered signals.
    lpf_behavior = np.zeros((behavior_traces.shape[0], behavior_traces.shape[1] - stim_idx))

    for i in range(behavior_traces.shape[0]):
        lpf_behavior[i, :] = butter_lowpass_filter(behavior_traces[i,stim_idx:], cutoff, fs, order)

    plt.subplot(2, 1, 2)
    plt.plot(np.mean(behavior_traces[:, stim_idx:], axis=0), 'b-', label='data')
    plt.plot(np.mean(lpf_behavior, axis=0), 'g-', linewidth=2, label='filtered data')
    # plt.plot(np.mean(lpf_behavior_QC, axis=0), 'r-', linewidth=2, label='QC filtered data')
    plt.xlabel('Time')
    # plt.grid()
    plt.legend()

    plt.subplots_adjust(hspace=0.35)
    plt.show()
    return lpf_behavior

In [ ]:
def get_speed_change(behavior, stim_idx, first_val_start_idx, first_val_end_idx, second_val_start_idx, second_val_end_idx):
    pre_tsi= first_val_start_idx-stim_idx
    pre_tei = first_val_end_idx-stim_idx
    post_tsi = second_val_start_idx-stim_idx
    post_tei = second_val_end_idx-stim_idx
    
    if first_val_start_idx!=first_val_end_idx:
        first_speed = np.mean(behavior[:,pre_tsi:pre_tei], axis = 1)
    else:
        first_val_start_idx = int(first_val_start_idx)
        first_speed = np.asarray(behavior[:,pre_tsi])

    if second_val_start_idx!=second_val_end_idx:
        second_speed = np.mean(behavior[:,post_tsi:post_tei], axis = 1)
    else: 
        second_speed = np.asarray(behavior[:,post_tsi])
    speed_change = second_speed-first_speed
    return speed_change

In [ ]:
def get_trials(speed_change, thresh):
    decrease_trials = speed_change <-thresh
    decrease_idx = np.where(decrease_trials)
    increase_trials = speed_change>thresh
    increase_idx = np.where(increase_trials)
    flat_trials = np.logical_and(speed_change>-thresh, speed_change<thresh)
    flat_idx = np.where(flat_trials)
    return increase_trials, decrease_trials, flat_trials

In [ ]:

def validate_trial(trial_idx, trace=None):
    """ Visualize a single trial, along with the pre and post averaging windows
    
    RED: pre window
    YELLOW: post window
    BLUE: stim time
    """
    if trace is None:
        trace = behavior_traces[trial_idx]
    plt.plot(trace)
    plt.axvline(first_val_start_idx, color='r')
    plt.axvline(first_val_end_idx, color='r')
    plt.axvline(second_val_start_idx, color='y')
    plt.axvline(second_val_end_idx, color='y')
    plt.axvline(stim_idx, color='b')


def validate_windows(behavior_traces, value_struct, fr):
    """ Visualize average traces for both slow and fast trials
    
    NOTE: uses global variables, must rerun constant cells and slow_ind logic to change results.
    
    RED: pre window
    YELLOW: post window
    BLUE: stim time
    """
    
    stim_idx, stim_end_idx, pre_trial_window_idx, first_val_end_idx, first_val_start_idx, second_val_end_idx, second_val_start_idx = get_vals_for_analysis(**value_struct)
    lpf_behavior = apply_butter_lowpass(behavior_traces, stim_idx, fr)
    speed_change = get_speed_change(lpf_behavior, stim_idx, first_val_start_idx, first_val_end_idx, second_val_start_idx, second_val_end_idx)
    increase_trials, decrease_trials, flat_trials = get_trials(speed_change, value_struct["thresh"])
    increase_traces = behavior_traces[increase_trials]
    decrease_traces = behavior_traces[decrease_trials]
    flat_traces = behavior_traces[flat_trials]
    
    print(f"{len(increase_traces)} increase trials")
    print(f"{len(decrease_traces)} decrease trials")
    print(f"{len(flat_traces)} flat trials")
    
    increase_mean = np.mean(increase_traces, axis=0)
    increase_sem = scipy.stats.sem(increase_traces, axis=0)
    
    decrease_mean = np.mean(decrease_traces, axis=0)
    decrease_sem = scipy.stats.sem(decrease_traces, axis=0)
    
    flat_mean = np.mean(flat_traces, axis=0)
    flat_sem = scipy.stats.sem(flat_traces, axis=0)
    
    fig, axes = plt.subplots(nrows=3)
    
    axes[0].plot(increase_mean, color='k',linewidth=3)
    axes[0].set_ylim(-1, 3.5)
    axes[0].fill_between(np.arange(len(increase_mean)), increase_mean - increase_sem, increase_mean + increase_sem, color='k',alpha=0.3)
    axes[0].axvline(first_val_start_idx, color='r')
    axes[0].axvline(first_val_end_idx, color='r')
    axes[0].axvline(second_val_start_idx, color='y')
    axes[0].axvline(second_val_end_idx, color='y')
    axes[0].axvline(stim_idx, color='b', linestyle='--')
    axes[0].axvline(stim_end_idx, color='b', linestyle='--')
#     axes[0].set_title("slow trials")
    

    
    axes[1].plot(decrease_mean, color='k',linewidth=3)
    axes[1].set_ylim(-1, 3.5)
    axes[1].fill_between(np.arange(len(decrease_mean)), decrease_mean - decrease_sem, decrease_mean + decrease_sem, color='k',alpha=0.3)
    axes[1].axvline(first_val_start_idx, color='r')
    axes[1].axvline(first_val_end_idx, color='r')
    axes[1].axvline(second_val_start_idx, color='y')
    axes[1].axvline(second_val_end_idx, color='y')
    axes[1].axvline(stim_idx, color='b', linestyle='--')
    axes[1].axvline(stim_end_idx, color='b', linestyle='--')
    
    axes[2].plot(flat_mean, color='k',linewidth=3)
    axes[2].set_ylim(-1, 3.5)
    axes[2].fill_between(np.arange(len(flat_mean)), flat_mean - flat_sem, flat_mean + flat_sem, color='k',alpha=0.3)
    axes[2].axvline(first_val_start_idx, color='r')
    axes[2].axvline(first_val_end_idx, color='r')
    axes[2].axvline(second_val_start_idx, color='y')
    axes[2].axvline(second_val_end_idx, color='y')
    axes[2].axvline(stim_idx, color='b', linestyle='--')
    axes[2].axvline(stim_end_idx, color='b', linestyle='--')
    
    return fig, axes


In [ ]:
fig, axes = validate_windows(behavior_traces_QC,value_struct, fr)
fig.tight_layout()

In [ ]:
fig, axes = validate_windows(behavior_traces, value_struct, fr)
fig.tight_layout()

# load neural

In [ ]:
n_clusters = 2000
cluster_dir = os.path.join(func_path, 'func_0','clustering')

load_file = os.path.join(cluster_dir, 'cluster_labels.npy')
cluster_labels = np.load(load_file)

load_file = os.path.join(cluster_dir, 'cluster_signals.npy')
all_signals = np.load(load_file)

In [ ]:
np.shape(all_signals)

In [ ]:
time_average =np.mean(all_signals, axis=(0,1))

# Fictrac checks

In [ ]:
fictrac['Y'] = np.squeeze(fictrac['Y'])
out = scipy.signal.correlate(time_average, fictrac['Y'])

In [ ]:
plt.plot(out)
plt.axvline(3384,color='k')
plt.xlim(3300,3400)

# check warp quality

In [ ]:
fixed = brainsss.load_fda_meanbrain()

In [ ]:
fixed

In [ ]:
atlas = brainsss.load_roi_atlas()

In [ ]:
explosion_rois = brainsss.load_explosion_groups()
all_rois = brainsss.unnest_roi_groups(explosion_rois)
roi_masks = brainsss.make_single_roi_masks(all_rois, atlas)
roi_contours = brainsss.make_single_roi_contours(roi_masks, atlas)

In [ ]:
timestamps = brainsss.load_timestamps(os.path.join(func_path,'func_0', 'imaging'))

In [ ]:
warp_dir = os.path.join(func_path, 'warp')
f2a = ants.image_read(os.path.join(warp_dir, 'func-to-anat.nii'))

In [ ]:
plt.imshow(f2a[:,:,70].T)

In [ ]:
a2a = ants.image_read(os.path.join(warp_dir, 'anat-to-meanbrain.nii'))

In [ ]:
a2a

In [ ]:
anat_dir = os.path.join(func_path, 'anat_0', 'moco')
anat = ants.image_read(os.path.join(anat_dir, 'anatomy_channel_1_moc_mean.nii'))

In [ ]:
anat.set_spacing((0.653, 0.653, 1))

In [ ]:
anat_low=anat.resample_image((2,2,2),use_voxels=False)

In [ ]:
np.shape(a2a)

In [ ]:
z=20
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.imshow(a2a[:,:,z].T)
plt.subplot(2,1,2)
plt.imshow(fixed[:,:,z].T)

# STA wrapped

In [ ]:
bin_start = -500; bin_end = 2000; bin_size = 100 #ms
neural_bins = np.arange(bin_start,bin_end,bin_size)

In [ ]:
####CHANGE BEHAVIOR!!!
# behavior_traces = behavior_traces
behavior_traces = behavior_traces_QC

In [ ]:
stim_idx, stim_end_idx, pre_trial_window_idx, first_val_end_idx, first_val_start_idx, second_val_end_idx, second_val_start_idx = get_vals_for_analysis(**value_struct)
lpf_behavior = apply_butter_lowpass(behavior_traces, stim_idx, fr)
speed_change = get_speed_change(lpf_behavior, stim_idx, first_val_start_idx, first_val_end_idx, second_val_start_idx, second_val_end_idx)
increase_trials, decrease_trials, flat_trials = get_trials(speed_change, value_struct["thresh"])

In [ ]:
def get_event_times_list(trials, event_times, ms=True):
    trials_idx = np.where(trials)
    trials_starts =[]
    for idx in trials_idx[0]:
        trials_starts.append(event_times[idx])
    if ms==True:
        trials_starts = [n*10 for n in trials_starts]
    print(np.shape(trials_starts))
    return trials_starts

In [ ]:
# # For regular data
# decrease_starts_ms = get_event_times_list(decrease_trials, actual_looms, ms=True)
# increase_starts_ms = get_event_times_list(increase_trials, actual_looms, ms=True)
# flat_starts_ms = get_event_times_list(flat_trials, actual_looms, ms=True)

# For QC data
decrease_starts_ms = get_event_times_list(decrease_trials, null_times, ms=True)
increase_starts_ms = get_event_times_list(increase_trials, null_times, ms=True)
flat_starts_ms = get_event_times_list(flat_trials, null_times, ms=True)

In [ ]:
t0 = time.time()
STA_brain_decrease = brainsss.make_STA_brain(neural_signals = all_signals,
                                   neural_timestamps = timestamps,
                                   event_times_list = decrease_starts_ms,
                                   neural_bins = neural_bins)

STA_brain_increase = brainsss.make_STA_brain(neural_signals = all_signals,
                                   neural_timestamps = timestamps,
                                   event_times_list = increase_starts_ms,
                                   neural_bins = neural_bins)

STA_brain_flat = brainsss.make_STA_brain(neural_signals = all_signals,
                                   neural_timestamps = timestamps,
                                   event_times_list = flat_starts_ms,
                                   neural_bins = neural_bins)
print(F'STA {time.time()-t0}')

In [ ]:
#### USE THIS FOR CORR

def make_STA_brain_corr(neural_signals, neural_timestamps, event_times_list, neural_bins):
    #### super voxel version

    STA_brain = []
    for z in range(49):
        print(z)
        ### interpolate behavior to neural data
        #https://github.com/ClandininLab/brainsss/blob/main/scripts/correlation.py
        behavior = 'dRotLabY'
        fictrac_interp = brainsss.smooth_and_interp_fictrac(fictrac_raw, fps, resolution, expt_len, behavior, timestamps=timestamps, z=z)
        
        all_bin_indicies = []
        for stim_idx in range(len(event_times_list)):
            stim_time = event_times_list[stim_idx]
            stim_centered_bins = neural_bins + stim_time
            bin_indicies = np.digitize(neural_timestamps[:,z] , stim_centered_bins)
            all_bin_indicies.append(bin_indicies)
        all_bin_indicies = np.asarray(all_bin_indicies)

        avg_neural_across_bins = []
        for bin_num in np.arange(1,len(neural_bins)):
            this_bin_sample_times = list(np.where(all_bin_indicies==bin_num)[1])
            
            neural_data_points = neural_signals[z,:,this_bin_sample_times]
            behavior_data_points = fictrac_interp[this_bin_sample_times]
            
            corr = []
            for cluster in range(2000):
                corr.append(scipy.stats.pearsonr(behavior_data_points, neural_data_points[:,cluster])[0])
            corr = np.asarray(corr)
            
            avg_neural_across_bins.append(corr)
        avg_neural_across_bins = np.asarray(avg_neural_across_bins)
        STA_brain.append(avg_neural_across_bins)
    STA_brain = np.asarray(STA_brain)
    return STA_brain

corr_brain = make_STA_brain_corr(neural_signals = all_signals,
                                   neural_timestamps = timestamps,
                                   event_times_list = actual_looms_ms, #### FIX THIS? ms.
                                   neural_bins = neural_bins)

In [ ]:
neural_signals = all_signals
neural_timestamps = timestamps
event_times_list = actual_looms_ms
neural_bins = neural_bins
            
z=20
### interpolate behavior to neural data
#https://github.com/ClandininLab/brainsss/blob/main/scripts/correlation.py
behavior = 'dRotLabY'
fictrac_interp = brainsss.smooth_and_interp_fictrac(fictrac_raw, fps, resolution, expt_len, behavior, timestamps=timestamps, z=z)

all_bin_indicies = []
for stim_idx in range(len(event_times_list)):
    stim_time = event_times_list[stim_idx]
    stim_centered_bins = neural_bins + stim_time
    bin_indicies = np.digitize(neural_timestamps[:,z] , stim_centered_bins)
    all_bin_indicies.append(bin_indicies)
all_bin_indicies = np.asarray(all_bin_indicies)

avg_neural_across_bins = []
for bin_num in np.arange(1,len(neural_bins)):
    this_bin_sample_times = list(np.where(all_bin_indicies==bin_num)[1])

    neural_data_points = neural_signals[z,:,this_bin_sample_times]
    behavior_data_points = fictrac_interp[this_bin_sample_times]

    corr = []
    for cluster in range(2000):
        corr.append(scipy.stats.pearsonr(behavior_data_points, neural_data_points[:,cluster])[0])
    corr = np.asarray(corr)

    avg_neural_across_bins.append(corr)
avg_neural_across_bins = np.asarray(avg_neural_across_bins)

In [ ]:
scipy.stats.pearsonr(behavior_data_points, neural_data_points[:,cluster])

In [ ]:
behavior_data_points.shape

In [ ]:
reformed_STA_brain_decrease = brainsss.STA_supervoxel_to_full_res(STA_brain_decrease, cluster_labels)


In [ ]:
n_clusters = 2000
n_tp = 24

colored_by_betas = np.zeros((n_tp, 256*128))
for cluster_num in range(n_clusters):
    cluster_indicies = np.where(cluster_labels[z,:]==cluster_num)[0]
    colored_by_betas[:,cluster_indicies] = avg_neural_across_bins[:,cluster_num,np.newaxis]
colored_by_betas = colored_by_betas.reshape(n_tp,256,128)


In [ ]:
plt.colored_by_betas.shape

In [ ]:
plt.imshow(colored_by_betas[7,:,:,].T)
plt.colorbar()

In [ ]:
avg_neural_across_bins.shape

In [ ]:
corr_brain.shape

In [ ]:
plt.hist(avg_neural_across_bins[7,:],bins=100);

In [ ]:
corr_brain = np.nan_to_num(corr_brain)
reformed_corr_brain = brainsss.STA_supervoxel_to_full_res(corr_brain, cluster_labels)

In [ ]:
STA_brain = gaussian_filter1d(reformed_corr_brain,sigma=1,axis=1,truncate=1)

In [ ]:
STA_brain_decrease = np.nan_to_num(STA_brain_decrease)
STA_brain_increase = np.nan_to_num(STA_brain_increase)
STA_brain_flat = np.nan_to_num(STA_brain_flat)

In [ ]:
reformed_STA_brain_decrease = brainsss.STA_supervoxel_to_full_res(STA_brain_decrease, cluster_labels)
reformed_STA_brain_increase = brainsss.STA_supervoxel_to_full_res(STA_brain_increase, cluster_labels)
reformed_STA_brain_flat = brainsss.STA_supervoxel_to_full_res(STA_brain_flat, cluster_labels)

In [ ]:
plt.imshow(reformed_STA_brain_decrease[25,15,:,:].T,)

In [ ]:
#change the STA brain based on condition
STA_brain = gaussian_filter1d(reformed_STA_brain_flat,sigma=1,axis=1,truncate=1)

In [ ]:
## STA_brain will be (bins,x,t,z) or (20,256,128,49)
#what you want now is (n,256,128,49), where n is number of looms

In [ ]:
def warp_STA_brain(STA_brain, fly, fixed, anat_to_mean_type, func_path):
    n_tp = STA_brain.shape[1]
    moving_resolution = (2.611, 2.611, 5)
    ###########################
    ### Organize Transforms ###
    ###########################
    warp_directory = os.path.join(func_path,'warp')
    warp_sub_dir = 'func-to-anat_fwdtransforms_2umiso'
    affine_file = os.listdir(os.path.join(warp_directory, warp_sub_dir))[0]
    affine_path = os.path.join(warp_directory, warp_sub_dir, affine_file)
    if anat_to_mean_type == 'myr':
        warp_sub_dir = 'anat-to-meanbrain_fwdtransforms_2umiso'
    elif anat_to_mean_type == 'non_myr':
        warp_sub_dir = 'anat-to-non_myr_mean_fwdtransforms_2umiso'
    else:
        print('invalid anat_to_mean_type')
        return
    syn_files = os.listdir(os.path.join(warp_directory, warp_sub_dir))
    syn_linear_path = os.path.join(warp_directory, warp_sub_dir, [x for x in syn_files if '.mat' in x][0])
    syn_nonlinear_path = os.path.join(warp_directory, warp_sub_dir, [x for x in syn_files if '.nii.gz' in x][0])
    ####transforms = [affine_path, syn_linear_path, syn_nonlinear_path]
    transforms = [syn_nonlinear_path, syn_linear_path, affine_path] ### INVERTED ORDER ON 20220503!!!!
    #ANTS DOCS ARE SHIT. THIS IS PROBABLY CORRECT, AT LEAST IT NOW WORKS FOR THE FLY(134) THAT WAS FAILING


    ### Warp timeponts
    warps = []
    for tp in range(n_tp):
        to_warp = np.rollaxis(STA_brain[:,tp,:,:],0,3)
        moving = ants.from_numpy(to_warp)
        moving.set_spacing(moving_resolution)
        ########################
        ### Apply Transforms ###
        ########################
        moco = ants.apply_transforms(fixed, moving, transforms)
        warped = moco.numpy()
        warps.append(warped)

    return warps

In [ ]:
t0 = time.time()
warps = warp_STA_brain(STA_brain=STA_brain, fly=fly_num, fixed=fixed, anat_to_mean_type='myr', func_path=func_path)
print(F'Warps {time.time()-t0}')

In [ ]:
def extract_roi_signal_traces(roi_ids, roi_masks, warps, hemi, signal_type):
    t0 = time.time()
    roi_time_avgs = []
    for roi in roi_ids[hemi]:
        mask = roi_masks[roi]
        masked_data = warps[:,:,:,:]*mask[np.newaxis,:,:,:] #note z-flip
        if signal_type == 'max':
            roi_time_avg = np.max(masked_data,axis=(1,2,3))
        elif signal_type == 'mean':
            roi_time_avg = np.mean(masked_data,axis=(1,2,3))
        roi_time_avgs.append(roi_time_avg)
    print(time.time()-t0)
    return np.asarray(roi_time_avgs)

In [ ]:
explosions = []
t0 = time.time()
for tp in range(24):
    input_canvas = np.ones((500,500,3)) #+.5 #.5 for diverging
    data_to_plot = warps[tp][:,:,::-1]
    vmax = 0.5 #this was 0.5 for STA <------------
    explosion_map = brainsss.place_roi_groups_on_canvas(explosion_rois,
                                                        roi_masks,
                                                        roi_contours,
                                                        data_to_plot,
                                                        input_canvas,
                                                        vmax=vmax,
                                                        cmap='seismic', diverging=True)#'hot')
    explosions.append(explosion_map)
print(F'Explosion {time.time()-t0}')

In [ ]:
i=0
plt.imshow(explosions[i][170:,:])

In [ ]:
neural_bins

In [ ]:
movie_dir = os.path.join(func_path, 'movies')
if os.path.exists(movie_dir)==False:
    os.mkdir(movie_dir)
print(movie_dir)

In [ ]:
save_dir = os.path.join(movie_dir,'flat_QC')
if os.path.exists(save_dir)==False:
    os.mkdir(save_dir)
print(save_dir)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(24):
    print(i)
    plt.imshow(explosions[i][170:,:]) #this was made with cmap=hot
    fname = os.path.join(save_dir, '{0:05d}.png'.format(i))
    plt.savefig(fname,dpi=300,bbox_inches='tight')

In [ ]:
def make_STA_brain(neural_signals, neural_timestamps, event_times_list, neural_bins):
    #### super voxel version
    
    STA_brain = []
    for z in range(49):
        all_bin_indicies = []
        for stim_idx in range(len(event_times_list)):
            stim_time = event_times_list[stim_idx]
            stim_centered_bins = neural_bins + stim_time
            bin_indicies = np.digitize(neural_timestamps[:,z] , stim_centered_bins)
            all_bin_indicies.append(bin_indicies)
        all_bin_indicies = np.asarray(all_bin_indicies)

        avg_neural_across_bins = []
        for bin_num in np.arange(1,len(neural_bins)):
            this_bin_sample_times = list(np.where(all_bin_indicies==bin_num)[1])
            average_neural_in_bin = np.mean(neural_signals[z,:,this_bin_sample_times],axis=0)
            avg_neural_across_bins.append(average_neural_in_bin)
        avg_neural_across_bins = np.asarray(avg_neural_across_bins)
        STA_brain.append(avg_neural_across_bins)
    STA_brain = np.asarray(STA_brain)
    return STA_brain

In [ ]:
for z in range(1): 
    all_bin_indicies = []
    for stim_idx in range(len(starts_loom_ms)):
        stim_time = starts_loom_ms[stim_idx]
        stim_centered_bins = neural_bins + stim_time
        bin_indicies = np.digitize(timestamps[:,z] , stim_centered_bins)
        all_bin_indicies.append(bin_indicies)
    all_bin_indicies = np.asarray(all_bin_indicies)

    avg_neural_across_bins = []
    for bin_num in np.arange(1,len(neural_bins)):
        this_bin_sample_times = list(np.where(all_bin_indicies==bin_num)[1])
        print(np.shape(this_bin_sample_times))
        average_neural_in_bin = np.mean(all_signals[z,:,this_bin_sample_times],axis=0)
        avg_neural_across_bins.append(average_neural_in_bin)
    avg_neural_across_bins = np.asarray(avg_neural_across_bins)

In [ ]:



behavior = 'dRotLabY'
fictrac_interp = brainsss.smooth_and_interp_fictrac(fictrac_raw, fps, resolution, expt_len, behavior, timestamps=timestamps, z=z)
    

In [ ]:
for z in range(1): 
    
    ### interpolate behavior to neural data
    #https://github.com/ClandininLab/brainsss/blob/main/scripts/correlation.py
    behavior = 'dRotLabY'
    fictrac_interp = brainsss.smooth_and_interp_fictrac(fictrac_raw, fps, resolution, expt_len, behavior, timestamps=timestamps, z=z)
    
    all_bin_indicies = []
    for stim_idx in range(len(starts_loom_ms)):
        stim_time = starts_loom_ms[stim_idx]
        stim_centered_bins = neural_bins + stim_time
        bin_indicies = np.digitize(timestamps[:,z] , stim_centered_bins)
        all_bin_indicies.append(bin_indicies)
    all_bin_indicies = np.asarray(all_bin_indicies)

    avg_neural_across_bins = []
    for bin_num in np.arange(1,len(neural_bins)):
        this_bin_sample_times = list(np.where(all_bin_indicies==bin_num)[1])
        print(np.shape(this_bin_sample_times))
        #average_neural_in_bin = np.mean(all_signals[z,:,this_bin_sample_times],axis=0)
        neural_data_points = all_signals[z,:,this_bin_sample_times]
        behavior_data_points = fictrac_interp[this_bin_sample_times]
        
        corr = scipy.stats.pearsonr(behavior_data_points, neural_data_points)[0]
        avg_neural_across_bins.append(corr)
    avg_neural_across_bins = np.asarray(avg_neural_across_bins)

In [ ]:
print(np.shape(timestamps))
print(np.shape(all_bin_indicies))

In [ ]:
t0 = time.time()
STA_brain = brainsss.make_STA_brain(neural_signals = all_signals,
                                   neural_timestamps = timestamps,
                                   event_times_list = starts_loom_ms,
                                   neural_bins = neural_bins)

print(F'STA {time.time()-t0}')